<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.3962385654449463

Current batch: 100
Current benign train accuracy: 0.15625
Current benign train loss: 2.144805669784546

Current batch: 200
Current benign train accuracy: 0.28125
Current benign train loss: 1.9223406314849854

Current batch: 300
Current benign train accuracy: 0.3203125
Current benign train loss: 1.8966960906982422

Current batch: 400
Current benign train accuracy: 0.34375
Current benign train loss: 1.738278865814209

Current batch: 500
Current benign train accuracy: 0.296875
Current benign train loss: 1.8979251384735107

Current batch: 600
Current benign train accuracy: 0.453125
Current benign train loss: 1.6652116775512695

Current batch: 700
Current benign train accuracy: 0.3828125
Current benign train loss: 1.6482309103012085

Current batch: 800
Current benign train accuracy: 0.421875
Current benign train loss: 1.637963056564331

Current batch: 900
Current ben


Current batch: 800
Current benign train accuracy: 0.8359375
Current benign train loss: 0.39863112568855286

Current batch: 900
Current benign train accuracy: 0.90625
Current benign train loss: 0.2635240852832794

Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3852404057979584

Current batch: 1100
Current benign train accuracy: 0.84375
Current benign train loss: 0.4594039022922516

Total benign train accuarcy: 84.998
Total benign train loss: 506.5210917145014

[ Test epoch: 5 ]

Test accuarcy: 81.96
Test average loss: 0.005527249048650265
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3293502628803253

Current batch: 100
Current benign train accuracy: 0.84375
Current benign train loss: 0.3870970606803894

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.44822821021080017

Current batch: 300
Current benign train accuracy: 0.8515625



Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.19871604442596436

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.2872019410133362

Current batch: 400
Current benign train accuracy: 0.875
Current benign train loss: 0.3367842435836792

Current batch: 500
Current benign train accuracy: 0.875
Current benign train loss: 0.35269221663475037

Current batch: 600
Current benign train accuracy: 0.875
Current benign train loss: 0.3285403251647949

Current batch: 700
Current benign train accuracy: 0.875
Current benign train loss: 0.3354054391384125

Current batch: 800
Current benign train accuracy: 0.875
Current benign train loss: 0.4258958399295807

Current batch: 900
Current benign train accuracy: 0.8359375
Current benign train loss: 0.39531946182250977

Current batch: 1000
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3693118393421173

Current batch: 1100
Current benign train accuracy: 0.89


Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2620126008987427

Current batch: 1100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.24453133344650269

Total benign train accuarcy: 91.022
Total benign train loss: 307.5859967917204

[ Test epoch: 16 ]

Test accuarcy: 84.68
Test average loss: 0.004950822451710701
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.20094147324562073

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2583998441696167

Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21122246980667114

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2178669273853302

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2625102996826172

Current batch: 500
Current benign train accuracy: 0.91


Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2255667895078659

Current batch: 500
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2663123905658722

Current batch: 600
Current benign train accuracy: 0.921875
Current benign train loss: 0.2561092972755432

Current batch: 700
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2039157897233963

Current batch: 800
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2830653786659241

Current batch: 900
Current benign train accuracy: 0.9453125
Current benign train loss: 0.17508870363235474

Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.4205714464187622

Current batch: 1100
Current benign train accuracy: 0.90625
Current benign train loss: 0.27762913703918457

Total benign train accuarcy: 91.594
Total benign train loss: 289.2900708094239

[ Test epoch: 22 ]

Test accuarcy: 85.45
Test average loss: 0.

...


Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0030280519276857376

Current batch: 800
Current benign train accuracy: 0.984375
Current benign train loss: 0.01819901540875435

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.010999688878655434

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.012078609317541122

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.011595437303185463

Total benign train accuarcy: 99.57266666666666
Total benign train loss: 16.855743088002782

[ Test epoch: 72 ]

Test accuarcy: 92.77
Test average loss: 0.0028873058304190635
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.01627877540886402

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0046507385559380054

Current batch: 200
Current benign train accuracy: 1.0
Curr


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0031267108861356974

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.011372794397175312

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.010320447385311127

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.01394509244710207

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.010540978983044624

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.009380892850458622

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.024699272587895393

Current batch: 800
Current benign train accuracy: 0.984375
Current benign train loss: 0.020266646519303322

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.01273263804614544

Current batch: 1000
Current benign train accuracy: 0.9921875
C


Current batch: 900
Current benign train accuracy: 0.984375
Current benign train loss: 0.025188501924276352

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0029712230898439884

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.007771025411784649

Total benign train accuarcy: 99.23666666666666
Total benign train loss: 27.865604266757146

[ Test epoch: 83 ]

Test accuarcy: 92.19
Test average loss: 0.0030701014541089536
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.023264162242412567

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0030410645995289087

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.006675925571471453

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.007499737199395895

Current batch: 400
Current benign train accuracy: 1.0
Cu


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.00966788362711668

Current batch: 400
Current benign train accuracy: 0.984375
Current benign train loss: 0.032686855643987656

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.011627651751041412

Current batch: 600
Current benign train accuracy: 0.984375
Current benign train loss: 0.053804874420166016

Current batch: 700
Current benign train accuracy: 0.984375
Current benign train loss: 0.0551585890352726

Current batch: 800
Current benign train accuracy: 0.9609375
Current benign train loss: 0.06664880365133286

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.015813905745744705

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.008558264002203941

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.012085719034075737

Total benign train accuarcy: 99.06
Total benign train


Total benign train accuarcy: 99.82933333333334
Total benign train loss: 7.750259630964138

[ Test epoch: 94 ]

Test accuarcy: 93.12
Test average loss: 0.002613907627761364
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.006021220237016678

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00243302877061069

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.005739434156566858

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.005225720815360546

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.005688399542123079

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.010460068471729755

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.00392382126301527

Current batch: 700
Current benign train accuracy: 1.0
Current benign 